In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import cv2
import os
import numpy as np
import pytesseract
import re

In [2]:
def noiseRemoval(image):
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    image = cv2.medianBlur(image, 3)
    return image

In [3]:
# Specify the desired image dimensions
image_height = 500
image_width = 500

# Path to the input image
image_path = "Data/78bce3b1.jpeg"

# Path to store the preprocessed image
preprocessed_path = "TestOne/78bce3b1.jpeg"

# Load the image
image = cv2.imread(image_path)

# Invert the image
inverted_image = cv2.bitwise_not(image)

# Rescale the image (optional)
rescaled_image = cv2.resize(inverted_image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

# Convert to grayscale
grayscale_image = cv2.cvtColor(rescaled_image, cv2.COLOR_BGR2GRAY)

# Binarization
thresholdImage, binary_image = cv2.threshold(grayscale_image, 127, 255, cv2.THRESH_BINARY)

# Noise removal (optional)
denoised_image = noiseRemoval(binary_image)

# Save the preprocessed image
cv2.imwrite(preprocessed_path, denoised_image)

True

In [4]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [10]:
# Perform text detection using Tesseract at multiple scales
scales = [1.0, 0.8, 0.6]  # Adjust the scales as per your requirement
for scale in scales:
    scaled_image = cv2.resize(denoised_image, None, fx=scale, fy=scale)

    # Perform text detection on the scaled image
    text_boxes = pytesseract.image_to_boxes(scaled_image, config='--psm 6')

    # Iterate over the detected text boxes
    for box in text_boxes.splitlines():
        # Extract coordinates and text information
        x, y, w, h, text = box.split(' ')[1:]
        x, y, w, h = int(x), int(y), int(w), int(h)

        # Rescale the bounding box coordinates to the original image scale
        x, y, w, h = int(x / scale), int(y / scale), int(w / scale), int(h / scale)

        # Draw bounding box around the text on the original image
        cv2.rectangle(image, (x, y), (w, h), (0, 255, 0), 2)

        # Display the text
        cv2.putText(image, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        # Use regex to search for total-related keywords
        if re.search(r'TOTAL|AMOUNT|BILL|BALANCE', text, re.IGNORECASE):
            print("Total-related Text:", text)

    # Perform text recognition using Tesseract on the original image
    recognized_text = pytesseract.image_to_string(image, config='--psm 6')

    # Print the recognized text
    print("Recognized Text:")
    print(recognized_text)
    print("-----------------------")

    # Get the first text element as merchant name
    first_text_box = text_boxes.splitlines()[0]
    x, y, w, h, text = first_text_box.split(' ')[1:]
    x, y, w, h = int(x), int(y), int(w), int(h)
    merchant_name = text

    # Print the detected merchant name
    print("Merchant Name:", merchant_name)
    print("-----------------------")

Recognized Text:
® g0 :
19 9 OO
ipo ™ th
MIE QO! én lit
mmr Peaaoor
he Caf
East Hampton, WA OQOOGO
alll) 3e9-t000 | wat Li
ax (631) 322-1 ot)
win oo ldenpearcafe MAbataed OM CU
bate: 7 hagoe' 15 11 MLL | OC
Card Type: Visa O aD O
Acct #: YOOXXAKAKXXK | oenll ‘A 1
Exp Date: — XX/XX u I
Auth Code: 08316C
Check: $981 qd
Check 10: CHINO ie
Server: 5 Karina I
U
Subtotal: 22.30
re —_—$B8P
Total ee, aod
ome a ee
gst
T agree to pay above total 0 ; ae
according to my card isWOOOOOGHY)  eeaiag
1 ligase
agreement UEWU00 ft a
Customer pec
= snc en
com copm00 F =“n=noqmo
HAL! wig mMpecay es cate

-----------------------
Merchant Name: 0
-----------------------
Recognized Text:
° g0 :
19 9 O98
ipo ™ th
MI CQO!C én tit
mmr Peaaar
he Caf
East Hampton, Wet OQOOGO
alll) 3e9-t000 un wat Li
ax (631) 22-1 ot)
wn oo ldenpearcafe MAeabaed OM CU
bate: 7 hagoe' 15 11 MLL | OC
Card Type: Visa 0 Wan 6
Acct #: YOOXXAKAKXXK | O11! fl 1
Exp Date: — XX/XX u I
Auth Code: 08316C
Check: $981 qd
Check 10: CHINO is
Serve